<a href="https://www.kaggle.com/code/vijayadithyabk/quora-spam-filtering?scriptVersionId=213274626" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
import re
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

In [2]:
# Load the data
data_path = '/kaggle/input/spam-filter-for-quora-questions/train.csv'
data = pd.read_csv(data_path)

In [3]:
# Inspect the dataset
print("Dataset Overview:")
print(data.head())
print("\nDataset Info:")
data.info()

Dataset Overview:
                    qid                                      question_text  \
0  00002165364db923c7e6  How did Quebec nationalists see their province...   
1  000032939017120e6e44  Do you have an adopted dog, how would you enco...   
2  0000412ca6e4628ce2cf  Why does velocity affect time? Does velocity a...   
3  000042bf85aa498cd78e  How did Otto von Guericke used the Magdeburg h...   
4  0000455dfa3e01eae3af  Can I convert montra helicon D to a mountain b...   

   target  
0       0  
1       0  
2       0  
3       0  
4       0  

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1306122 entries, 0 to 1306121
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   qid            1306122 non-null  object
 1   question_text  1306122 non-null  object
 2   target         1306122 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 29.9+ MB


In [4]:
# Preprocess the data
def clean_text(text):
    text = re.sub(r'[^a-zA-Z0-9 ]', '', text)  # Remove special characters
    text = text.lower()  # Convert to lowercase
    return text

data['question_text'] = data['question_text'].apply(clean_text)

In [5]:
# Extract features and labels
X = data['question_text']
y = data['target']  # Assuming 'target' column contains the labels

In [6]:
# Train-validation split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# Tokenize and pad sequences
max_words = 20000  # Vocabulary size
max_len = 100  # Maximum length of sequences

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_val)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_val_pad = pad_sequences(X_val_seq, maxlen=max_len)

In [8]:
# !wget http://nlp.stanford.edu/data/glove.42B.300d.zip

In [9]:
# !unzip /kaggle/working/glove.42B.300d.zip
# !rm /kaggle/working/glove.42B.300d.zip

In [10]:
# Load GloVe embeddings
glove_path = '/kaggle/input/glove-42b-300de/glove.42B.300d.txt'  # Replace with your GloVe file path
embedding_dim = 100

embeddings_index = {}
with open(glove_path, 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print(f"Loaded {len(embeddings_index)} word vectors from GloVe.")

Loaded 1917494 word vectors from GloVe.


In [11]:
# Create embedding matrix
word_index = tokenizer.word_index
# Update the embedding dimension to match the GloVe file (e.g., 300)
embedding_dim = 300

# Create embedding matrix
embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector


In [12]:
# Build the model
model = Sequential([
    Embedding(input_dim=max_words, 
              output_dim=embedding_dim, 
              weights=[embedding_matrix], 
              input_length=max_len, 
              trainable=False),
    LSTM(128, dropout=0.2, recurrent_dropout=0.2),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print(model.summary())

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │     6,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,000,000 (22.89 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 6,000,000 (22.89 MB)

None


In [13]:
# Train the model
batch_size = 32
epochs = 5

history = model.fit(
    X_train_pad, y_train,
    validation_data=(X_val_pad, y_val),
    batch_size=batch_size,
    epochs=epochs,
    verbose=1
)

Epoch 1/5
32654/32654 ━━━━━━━━━━━━━━━━━━━━ 4027s 123ms/step - accuracy: 0.9501 - loss: 0.1318 - val_accuracy: 0.9572 - val_loss: 0.1073
Epoch 2/5
32654/32654 ━━━━━━━━━━━━━━━━━━━━ 4073s 125ms/step - accuracy: 0.9567 - loss: 0.1096 - val_accuracy: 0.9590 - val_loss: 0.1041
Epoch 3/5
32654/32654 ━━━━━━━━━━━━━━━━━━━━ 4113s 126ms/step - accuracy: 0.9584 - loss: 0.1050 - val_accuracy: 0.9596 - val_loss: 0.1033
Epoch 4/5
32654/32654 ━━━━━━━━━━━━━━━━━━━━ 4119s 126ms/step - accuracy: 0.9589 - loss: 0.1032 - val_accuracy: 0.9598 - val_loss: 0.1027
Epoch 5/5
32654/32654 ━━━━━━━━━━━━━━━━━━━━ 4130s 126ms/step - accuracy: 0.9599 - loss: 0.1012 - val_accuracy: 0.9596 - val_loss: 0.1041


In [14]:
# Evaluate the model
val_pred = (model.predict(X_val_pad) > 0.5).astype(int)
print("Classification Report:")
print(classification_report(y_val, val_pred))

8164/8164 ━━━━━━━━━━━━━━━━━━━━ 338s 41ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.98      0.98    245369
           1       0.69      0.61      0.65     15856

    accuracy                           0.96    261225
   macro avg       0.83      0.79      0.81    261225
weighted avg       0.96      0.96      0.96    261225

